In [ ]:
# helper functions
def show_python_code_and_result(response):
    for i in range(len(response)):
        # check if the response contains a valid function call result from the code executor
        if (
            (response[i].content.parts)
            and (response[i].content.parts[0])
            and (response[i].content.parts[0].function_response)
            and (response[i].content.parts[0].function_response.response)
        ):
            response_code = response[i].content.parts[0].function_response.response
            if "result" in response_code and response_code["result"] != "```":
                if "tool_code" in response_code["result"]:
                    print(
                        "Generated Python Code >> ",
                        response_code["result"].replace("tool_code", ""),
                    )
                else:
                    print("Generated Python Response >> ", response_code["result"])
print("Helper functions defined.")

✅ Helper functions defined.


In [ ]:
# retry options to handle errors like rate limits or temporary service unavailablity
retry_config = types.HttpRetryOptions(
    attempts=5,  # maxretry attempts
    exp_base=7,  # delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],  # retry on these http errors
)

<img src="https://storage.googleapis.com/github-repo/kaggle-5days-ai/day2/enhanced-currency-agent.png" width="800" alt="Enhanced Currency Converter Agent">

In [ ]:
# using built in code executor
calculation_agent = LlmAgent(
    name="CalculationAgent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""You are a specialized calculator that ONLY responds with Python code. You are forbidden from providing any text, explanations, or conversational responses.
 
     Your task is to take a request for a calculation and translate it into a single block of Python code that calculates the answer.
     
     **RULES:**
    1.  Your output MUST be ONLY a Python code block.
    2.  Do NOT write any text before or after the code block.
    3.  The Python code MUST calculate the result.
    4.  The Python code MUST print the final result to stdout.
    5.  You are PROHIBITED from performing the calculation yourself. Your only job is to generate the code that will perform the calculation.
   
    Failure to follow these rules will result in an error.
       """,
    code_executor=BuiltInCodeExecutor(),  # uses the built-in code executor Tool which gives the agent code execution capabilities
)

In [ ]:
enhanced_currency_agent = LlmAgent(
    name="enhanced_currency_agent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""You are a smart currency conversion assistant. You must strictly follow these steps and use the available tools.

  For any currency conversion request:

   1. Get Transaction Fee: Use the get_fee_for_payment_method() tool to determine the transaction fee.
   2. Get Exchange Rate: Use the get_exchange_rate() tool to get the currency conversion rate.
   3. Error Check: After each tool call, you must check the "status" field in the response. If the status is "error", you must stop and clearly explain the issue to the user.
   4. Calculate Final Amount (CRITICAL): You are strictly prohibited from performing any arithmetic calculations yourself. You must use the calculation_agent tool to generate Python code that calculates the final converted amount. This 
      code will use the fee information from step 1 and the exchange rate from step 2.
   5. Provide Detailed Breakdown: In your summary, you must:
       * State the final converted amount.
       * Explain how the result was calculated, including:
           * The fee percentage and the fee amount in the original currency.
           * The amount remaining after deducting the fee.
           * The exchange rate applied.
    """,
    tools=[
        get_fee_for_payment_method,
        get_exchange_rate,
        AgentTool(agent=calculation_agent),  # using another agent as a tool
    ],
)

print("Enhanced currency agent created")
print("New capability: Delegates calculations to specialist agent")
print("Tool types used:")
print("  • Function Tools (fees, rates)")
print("  • Agent Tool (calculation specialist)")

✅ Enhanced currency agent created
🎯 New capability: Delegates calculations to specialist agent
🔧 Tool types used:
  • Function Tools (fees, rates)
  • Agent Tool (calculation specialist)


In [17]:
# Define a runner
enhanced_runner = InMemoryRunner(agent=enhanced_currency_agent)

In [18]:
# Test the enhanced agent
response = await enhanced_runner.run_debug(
    "Convert 1,250 USD to INR using a Bank Transfer. Show me the precise calculation."
)


 ### Created new session: debug_session_id

User > Convert 1,250 USD to INR using a Bank Transfer. Show me the precise calculation.


enhanced_currency_agent > The final amount is 103159.5 INR.

Here's the breakdown of the calculation:

1.  **Fee Calculation**: A 1% transaction fee was applied to the original amount.
    *   Fee Percentage: 1.0%
    *   Fee Amount: 12.5 USD (1% of 1250 USD)

2.  **Net Amount**: The fee was deducted from the original amount.
    *   Amount after fee: 1237.5 USD (1250 USD - 12.5 USD)

3.  **Currency Conversion**: The net amount was converted to INR using the prevailing exchange rate.
    *   Exchange Rate: 83.58 INR per USD
    *   Final Amount in INR: 103159.5 INR (1237.5 USD \* 83.58 INR/USD)


In [19]:
show_python_code_and_result(response)

Generated Python Response >>  ```python
usd_amount = 1250
exchange_rate = 83.58
fee_percentage = 0.01

fee_amount = usd_amount * fee_percentage
net_usd_amount = usd_amount - fee_amount
inr_amount = net_usd_amount * exchange_rate

print(f"Original USD Amount: {usd_amount}")
print(f"Exchange Rate (USD to INR): {exchange_rate}")
print(f"Fee Percentage: {fee_percentage*100}%")
print(f"Fee Amount (USD): {fee_amount}")
print(f"Net USD Amount after Fee: {net_usd_amount}")
print(f"Final Amount in INR: {inr_amount}")
```
